In [ ]:
import numpy as np 
import pandas as pd 
import os
import itertools

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Input, merge, UpSampling2D, Cropping2D, ZeroPadding2D, Reshape, core, Convolution2D, Conv2DTranspose, Concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras import optimizers
from keras import backend as K
from keras.optimizers import SGD
from keras.layers.merge import concatenate
from keras.preprocessing.image import array_to_img
from keras.preprocessing.image import load_img

from tensorflow.python.keras.callbacks import TensorBoard

from sklearn.metrics import fbeta_score
from sklearn.model_selection import train_test_split

from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

import utils
from model_03 import get_model
import pickle

import keras.backend as K
import tensorflow as tf

from keras.models import load_model
import losses
import imageio

In [ ]:
files = utils.get_files("data/split")
print(len(files))

In [ ]:
np.random.shuffle(files)
#files = files[0:10]
n_valid = int(np.ceil(len(files) * 0.15))
n_test = int(np.ceil((len(files) - n_valid)* 0.15))
n_train = len(files) - n_valid - n_test
f_train = files[0:n_train]
f_valid = files[n_train: n_train + n_valid]
f_test = files[n_train + n_valid:]

In [ ]:
print(len(f_train))
print(len(f_test))
print(len(f_valid))

In [ ]:
X_train, Y_train = utils.load_data(f_train)
X_valid, Y_valid = utils.load_data(f_valid)

In [ ]:
model = get_model(img_size=512, n_kernels=3)
print(model.summary())

In [ ]:
model.compile(loss=losses.make_bce_loss(0.9971, 0.0029),
              optimizer=optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.98, nesterov=True),
              metrics=['accuracy'])
#K.get_session().run(tf.global_variables_initializer())
#tensorboard = TensorBoard(log_dir="logs") 
callbacks = [
             EarlyStopping(monitor='val_loss',
                           patience=4,
                           verbose=1),
             ModelCheckpoint("model07.h5",
                             monitor='val_loss',
                             save_best_only=True,
                             verbose=1)]

In [ ]:
# batch_size determines the number of samples in each mini batch
# steps_per_epoch the number of batch iterations before a training epoch is considered finished.
# validation_steps
hist = model.fit(x=X_train,
                 y=Y_train,
                 epochs=10,
                 verbose=2, callbacks=callbacks,
                 validation_data=(X_valid, Y_valid))

In [ ]:
training_loss = hist.history['loss']
test_loss = hist.history['val_loss']
training_acc = hist.history['acc']
test_acc = hist.history['val_acc']

epoch_count = range(1, len(training_loss) + 1)

plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Validation Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show();

plt.plot(epoch_count, training_acc, 'r--')
plt.plot(epoch_count, test_acc, 'b-')
plt.legend(['Training Acc', 'Validation Acc'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show();

In [ ]:
X_test, Y_test = preprocess.load_split_data(f_test)

In [ ]:
Y_pred = model.predict(X_test, batch_size=1, verbose=1)
Y_round = np.around(Y_pred)

for i in range(Y_pred.shape[0]):
    preprocess.show_ndimg(X_test[i], Y_test[i])
    preprocess.show_ndimg(Y_pred[i], Y_round[i])

In [ ]:
n = 10
Y_pred = model.predict(X_train[0:n], batch_size=1, verbose=1)
Y_round = np.around(Y_pred)

for i in range(Y_pred.shape[0]):
    preprocess.show_ndimg(X_train[i], Y_train[i])
    preprocess.show_ndimg(Y_pred[i], Y_round[i])

In [ ]:

n_total = np.prod(Y_train[0].shape)
n_ones = np.sum(Y_train[0])
n_zeros = n_total - n_ones
print(n_total)
print(n_ones)
print(n_zeros)
w_zero = n_ones / n_total
w_ones = n_zeros / n_total
print(w_zero)
print(w_ones)
print(w_zero + w_ones)
print(1 - 0.0022)

In [ ]:
Y_train[0]

In [ ]:
for i in range(Y_pred.shape[0]):
    preprocess.show_ndimg(X_test[i], Y_test[i])
    preprocess.show_ndimg(Y_pred[i], Y_round[i])